The problems with the spin up might be solved by fixing the negative moisture points. One simple way to do this is to implement the time stepper in a transformed space. There are several ways to do this, includiong

1. log transformation
2. power transformation.

In this notebook, I try log-transforming the inputs

In [ ]:
%matplotlib inline
import holoviews as hv
hv.extension('bokeh')
from IPython.display import Image

In [ ]:
ds = xr.open_dataset("../data/processed/training.nc")
location = ds.isel(x=0, y=5)

# Negative humidity from stepping with FQT

In [ ]:
dt = .125*86400
p = 2

state0 = location.QT
xform = np.power(np.abs(state0), 1/p)
xform1 = xform + location.FQT / p / np.power(np.abs(location.QT), (p-1)/p)   * dt
xform1 = xform1.where(location.QT > 1e-8, other=0.0)
state1 = np.power(np.abs(xform1), p)

In [ ]:
state1.T.plot()
plt.xlim([100, 110])

In [ ]:
state0.T.plot()
plt.xlim([100, 110])

Compared to time stepping with the other forcing

In [ ]:
state1_q_space = (state0 + dt * location.FQT)

Here are the points where the humidity after the large-forcings is negative.

In [ ]:
(state1_q_space < 0).plot(x='time')
plt.xlim([100, 110])

# Log-normalization of humidity

In [ ]:
q0 = ds.QT[0].quantile(.99,['x', 'y'])/1
q0 = q0.where(q0 > 1e-7)

In [ ]:
q0.plot(y='z')

Here is the humidity in the upper atmosphere:

In [ ]:
%opts Image[width=500, height=250, colorbar=True](cmap='viridis') {+framewise}
hv.Dataset(ds.QT[0]).to.image(["x", "y"])

The variance of points in the upper atmosphere is extreme. This suggest we could take a log transform:

$$ y = log(q_T/ q_0) + 1 $$

Where $q_0$ is the 99% quantile of the humidity at each level.

This transformation can be converted into a humidity tendency using $\dot{y} = \dot{q_T} / (q_T / q_0) $, so that $\dot{q_T} = y_t (q_T / q_0)$.

In [ ]:
qt_norm = np.log(ds.QT[0]/q0) + 1

hv.Dataset(qt_norm).to.image(["x", "y"])

In [ ]:
qt_norm[:,32].plot(x='x')

Maybe relative humidity is a better normalizer.

# Training Neural networks with the normalized humidities

## Predict log-normalized humidities (Run #134)


In [ ]:
Image("../models/134/fqtnn-obs2-8.png")

In [ ]:
Image("../models/134/fqtnn2-8.png")

Again we see a significant spin-up problem even when predicting with the log humidity fields. FWIW...what does the answer look like with seq_length=1.

## Log-normalized humidities with seq_length=1 (#135)

The FQTNN looks pretty similar to the time series above. Is there something wrong with the second plot in the previous section.

In [ ]:
Image("../models/135/fqtnn-obs3-8.png")

## Log normalized time series

What does the single column model time series of the transformed humidity look like?

In [ ]:
Image("../models/137/qt2-8.png")

Here is the actual time series:

In [ ]:
qt_norm = np.log(ds.QT[:,:,32,0]/q0) + 1

qt_norm.plot(x='time', vmax=6)
plt.xlim([100, 124])

There is some broad agreement between this simulation, but I do not think the log normalization trick works that well. What about just normalizing the data?

## Scaling only, no log

What if I normalize the data with $q \leftarrow q/q_0$, where $q_0$ is defined in the same way as above.

In [ ]:
Image("../models/138/fqtnn-obs2-8.png")

In [ ]:
Image("../models/138/fqtnn2-8.png")

the time series is a mess though

In [ ]:
Image("../models/138/qt2-8.png")